# DistributedBirding

# Description
Given a list of names and street addresses, 

For each person:
- create an individual species list
- create a list of hotspots
- Create a list of species that is unique to their 5MR (if any)

Note: Assumes roughly 5-20 contacts. More will work but will tax eBird and Nominatim, plus there are some n-squared algorithms involved.

## References
Geocode with Python: How to Convert physical addresses to Geographic locations → Latitude and Longitude, Abdishakur
https://towardsdatascience.com/geocode-with-python-161ec1e62b89

## Installation
pip install geopandas  
pip install geopy  
pip install tqdm
pip install xlrd

# Environment

## Library Imports

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
import re
import json
import requests
import math
import more_itertools
import urllib3
import typing

import traceback

from more_itertools import flatten

import geopandas as gpd

# import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

from tqdm import tqdm

# For making map
import geojson
import folium
from folium import plugins
from geojson_rewind import rewind
import webcolors

# For making sample data for article
import shapely
from shapely.geometry import Point

# For hotspot circles
import geog

from shapely.ops import nearest_points

# For downloading shapes file
from zipfile import ZipFile 

## Local Imports

In [ ]:
import xutilities
import xruxidownload

## Constants and Globals

In [ ]:
# Constants and Globals
DEBUGGING = False

DEFAULT_CIRCLE_RADIUS = 5 # Miles
KM_PER_MILE = 1.60934
MY5MR_PREFIX = 'my5mr-'
DEFAULT_MAX_OBSERVERS = 2
DEFAULT_DAYS_BACK = 14
MAX_DAYS_BACK = 30

EXTENSION_CSV = '.csv'
EXTENSION_EXCEL = '.xlsx'

NOMINATIM_USER_AGENT = 'DistributedBirding'

# Example of things we might want to filter out
# ['gull sp.',
#  'Tree/Violet-green Swallow',
#  'swallow sp.',
#  'hummingbird sp.',
#  "Rufous/Allen's Hummingbird",
#  'warbler sp. (Parulidae sp.)',
#  "Sharp-shinned/Cooper's Hawk",
#  'hawk sp.',
#  'woodpecker sp.',
#  'Downy/Hairy Woodpecker',
#  'Mallard (Domestic type)']

DEFAULT_SPECIES_FILTER = ['.sp'] # or for example, ['.sp', '\(', '/'] to eliminate all examples above

## Debug Section

In [ ]:
# https://api.ebird.org/v2/ref/hotspot/{{regionCode}}
# location = 'US-CA-085,US-CA-081' # Santa Clara County, San Mateo County
REGION_SANTA_CLARA_COUNTY = 'US-CA-085'
REGION_SAN_MATEO_COUNTY = 'US-CA-081'

## Jupyter-specific Imports and Settings

In [ ]:
# Data manipulation
# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython import get_ipython
ipython = get_ipython()

# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    get_ipython().run_line_magic('load_ext', 'autoreload')

get_ipython().run_line_magic('autoreload', '2')

## File Paths

In [ ]:
# https://medium.com/@rrfd/cookiecutter-data-science-organize-your-projects-atom-and-jupyter-2be7862f487e
# Base Path
base_path = Path.cwd()

# Data paths
data_path = base_path / 'data'
raw_data_path = data_path / 'raw'
interim_data_path = data_path / 'interim'
processed_data_path = data_path / 'processed'
external_data_path = data_path / 'external'

# Reports paths
reports_path = base_path / 'reports'
figures_path = reports_path / 'figures'

# Input paths
contacts_name = 'Contacts'
# We will append either '.csv' or '.xlsx' to contacts_base_path
contacts_base_path = external_data_path

# The Participants file defaults to the contacts file
# If a reduced set of participants is desired, create a file with a Name column
participants_base_path = external_data_path

# The Parameters file has 3 columns: RegionCodes, Radius, MaxObservers
# The name column is the 
parameters_name = 'Parameters'
parameters_base_path = external_data_path

# External data files for US County shape files and US State FIPS codes
# If these are not found, a Parameters file is required with a list of region codes
county_map_data_path = processed_data_path / 'tl_2017_us_county'
county_shx_shapes = 'tl_2017_us_county.shx'
county_shx_path = county_map_data_path / county_shx_shapes
state_fips_master_path = external_data_path / 'state_fips_master.csv'

# Interim paths
# address_geocache_path = interim_data_path / 'address_geocache.csv'
geolocation_cache_path = interim_data_path / 'GeolocationCache.csv'

# Outputs paths
unique_species_path = reports_path / f'{MY5MR_PREFIX}uniques.csv'
sharpies_map_path = reports_path / f'{MY5MR_PREFIX}team_circles.html'
hotspots_path = reports_path / f'{MY5MR_PREFIX}hotspots.csv'
unique_hotspots_path = reports_path / f'{MY5MR_PREFIX}unique-hotspots.csv'
sample_unique_species_path = reports_path / f'{MY5MR_PREFIX}sample_uniques.csv'
updated_contacts_path = reports_path / f'{MY5MR_PREFIX}updated_contacts.csv'
limited_path  = reports_path / f'{MY5MR_PREFIX}limited.csv'

# Shape file paths
shape_zip_path = raw_data_path / 'tl_2017_us_county.zip'
shape_dest_dir = processed_data_path / 'tl_2017_us_county'
        
# Credentials
eBirdCredential_path = '/Volumes/TSecure3/other/eBirdCredentials.yml'

# Code

In [ ]:
def create_project_paths():
    default_mode = 0o755
    data_path.mkdir(mode=default_mode, parents=False, exist_ok=True)
    raw_data_path.mkdir(mode=default_mode, parents=False, exist_ok=True)
    interim_data_path.mkdir(mode=default_mode, parents=False, exist_ok=True)
    processed_data_path.mkdir(mode=default_mode, parents=False, exist_ok=True)
    external_data_path.mkdir(mode=default_mode, parents=False, exist_ok=True)
    reports_path.mkdir(mode=default_mode, parents=False, exist_ok=True)
#     figures_path.mkdir(mode=default_mode, parents=False, exist_ok=True)

In [ ]:
# Manual way to find region:
# Go to https://ebird.org/explore
# Enter your county in the search field for "Explore Regions"
# When the page opens, look at the URL. Your region is after "region/" and before "?", e.g. US-CA-085
# https://ebird.org/region/US-CA-085?yr=all

# A parameters file is not required if the regions can be deduced from the contacts list
# This requires the US States shapes files and the US States FIPS code file
def read_parameters(parameters_path = parameters_base_path) -> pd.DataFrame:
    df = pd.DataFrame() # Failure mode

    try:
        # full path passed in 
        if parameters_path.is_file():
            if parameters_path.suffix == EXTENSION_CSV:
                return pd.read_csv(parameters_path, dtype=str).fillna('')
            else:
                xdtypes = {'Participants':str,'Region':str,
                           'Radius':float, 'MaxObservers':int, 'DaysBack':int}
                return pd.read_excel(parameters_path, header=0, dtypes=xdtypes).fillna('')
        else:
            csv_path = parameters_path / f'{parameters_name}{EXTENSION_CSV}'
            excel_path = parameters_path / f'{parameters_name}{EXTENSION_EXCEL}'

            if csv_path.exists():
                print(f'Parameters from: {csv_path}')
                df = pd.read_csv(csv_path, dtype=str).fillna('')
            elif excel_path.exists():
                print(f'Parameters from: {excel_path}')
                df = pd.read_excel(excel_path, header=0).fillna('')
            else:
                pass
            
    except Exception as ee:
        pass
    
    return df

In [ ]:
def miles_to_kilometers(mi: float):
    return mi * KM_PER_MILE

def get_recent_nearby_observations(latitude: float, longitude: float, radius: int = 9, back: int = 30):
    # https://ebird.org/ws2.0/ref/hotspot/geo?lat=37.4407&lng=-122.0936&fmt=json&dist=6
    recentobs = pd.DataFrame()
    try:
        if not math.isnan(latitude) and not math.isnan(longitude):
            api_url_base = 'https://ebird.org/ws2.0/data/obs/geo/recent'
            api_auth_header = {'X-eBirdApiToken': ebird_api_key}
            params = {
                'lat': str(latitude),
                'lng': str(longitude),
                'dist': str(radius),
                'back': str(back),
                'hotspot': 'True', # Should we limit only to hotspots?
                'includeProvisional': 'True' 
            }

            rr = requests.get(api_url_base, params=params, headers=api_auth_header, stream=True)
            if rr.status_code == requests.codes.ok:
                recentobs = pd.DataFrame(rr.json())
            rr.raise_for_status()

    except Exception as ee:
        print(ee)

    return recentobs


def get_observations(df, radius_miles = DEFAULT_CIRCLE_RADIUS, days_back = DEFAULT_DAYS_BACK):
    radius_kms = miles_to_kilometers(radius_miles)

    obs = []
    print(f"\nObservations by all eBird users in individual's {radius_miles:.0f}MR\n in past {days_back} days")
    print(f'{"Name":^30}{"Location":^30}{"Observations":>14}')          
          
    for index, row in df.iterrows():
        try:
            # assumes lat, lng exist
            ro = get_recent_nearby_observations(float(row['latitude']), float(row['longitude']), radius_kms, days_back)
            # Filter out some things
            search_values = DEFAULT_SPECIES_FILTER

            ro = ro[~ro.comName.str.contains('|'.join(search_values), case=True)]
          
            name, lat, lng = [row[col] for col in ['Name', 'latitude', 'longitude']]
            obscount = ro.shape[0]
            location = f'({lat:.5f}, {lng:.5f})'
            print(f'{name:<30}{location:<30}{obscount:>14}')

            obs.append(ro)
        except Exception as ee:
            print('get_observations', ee)
            obs.append(pd.DataFrame())

    print('')

    return df.assign(observations=obs) # This is a new dataframe

In [ ]:
# Read in CSV file with names and addresses
# Original columns in CSV file are:
#     ['Name', 'Street1', 'Street2', 'City', 'State', 'Zip', 'Cell', 'Email', 'latitude', 'longitude']
# although only ['Name', 'Street1', 'City', 'State', 'Zip', 'latitude', 'longitude'] are used right now

def load_contacts(contacts_path):
    contacts_csv_path, contacts_excel_path = contacts_base_path, contacts_base_path
    # If a full path to the file is given, use that
    if contacts_path.is_file():
        if contacts_path.suffix == EXTENSION_CSV:
            contacts_csv_path = contacts_path
            contacts_excel_path = None
        else:
            # may still fail if weird file passed in
            contacts_excel_path = contacts_path
            contacts_csv_path = None
    else:
        contacts_csv_path = contacts_path / f'{contacts_name}{EXTENSION_CSV}'
        contacts_excel_path = contacts_path / f'{contacts_name}{EXTENSION_EXCEL}'

    df = None # Failure mode
    if contacts_csv_path and contacts_csv_path.exists():
        print(f'CSV: {contacts_csv_path}')
        df = pd.read_csv(contacts_csv_path, dtype=str).fillna('')
        df['Address'] = df[['Street1', 'City', 'State', 'Zip']].agg(' '.join, axis=1).apply(str.strip)
    elif contacts_excel_path and contacts_excel_path.exists():
        # Excel imports Zip Codes as numeric
        print(f'Excel: {contacts_excel_path}')
        xdtypes = {'Name':str, 'Zip':str, 'latitude':float, 'longitude':float }
        df = pd.read_excel(contacts_excel_path, header=0, dtype=xdtypes).fillna('')
        df['Zip'] = df['Zip'].astype(str)
        df['Address'] = df[['Street1', 'City', 'State', 'Zip']].agg(' '.join, axis=1).apply(str.strip)
    else:
        raise UserWarning(f'No contacts file found in: {contacts_path}')
        
    return df


def load_and_update_contacts(contacts_path = contacts_base_path):
    contacts = load_contacts(contacts_path)

    contacts.latitude  = contacts.latitude.replace('', 0.0).astype(float, errors='ignore')
    contacts.longitude = contacts.longitude.replace('', 0.0).astype(float, errors='ignore')

    # These are the rows that need updating with geolocation information
    # No birding on Null Island; a (0,0) means we have no geo info
    update_mask = contacts.apply(lambda x: bool(x.Address.strip()) and (x.latitude + x.longitude == 0.0), axis=1)
    
    needsloc = contacts[update_mask].copy()

    if not needsloc.empty:
        newlocs = addresses_to_geolocations(needsloc.Address)

        merged = pd.merge(contacts, newlocs, how='left', left_on="Address", right_on="Address")
        for col in ['latitude', 'longitude']:
            merged[col + '_x'] = merged[col + '_x'].replace('', '0.0').astype(float)
            merged[col + '_y'] = merged[col + '_y'].replace(np.nan, 0.0).astype(float)
            merged[col] = merged[col + '_x'] + merged[col + '_y']

        contacts = merged.drop(['latitude_x', 'latitude_y', 'longitude_x', 'longitude_y'], axis=1)

    contacts['latitude']  = contacts['latitude'].replace(np.nan, 0.0).astype(float)
    contacts['longitude'] = contacts['longitude'].replace(np.nan, 0.0).astype(float)

    return contacts


def filter_participants(contacts, participants):
    df = contacts[contacts.Name.isin(participants)] if participants else contacts
    # We can't do much with out geocoords, so drop
    drop_mask = df.apply(lambda x: (x.latitude + x.longitude == 0.0), axis=1)
    no_coords = [x for x in list(df[drop_mask].Name) if x!='']
    if no_coords:
        print(f'No coordinates for: {", ".join(no_coords)}')
    return df[~drop_mask]

In [ ]:
def read_cached_geolocations():
    # Address, latitude, longitude
    cached_geolocations = pd.DataFrame()
    try:
        if geolocation_cache_path.is_file():
            cached_geolocations = pd.read_csv(geolocation_cache_path, index_col=False)
    except Exception as ee:
        print(ee)
        pass
    
    return cached_geolocations.fillna('')


# Don't call this directly -- use the cache when ever possible by calling addresses_to_geolocations
def find_geolocations(addresses: list):
    df = pd.DataFrame(addresses, columns=['Address'])
    delay_seconds = 2.0
    est_seconds = delay_seconds * len(df.index)

    locator = Nominatim(user_agent = NOMINATIM_USER_AGENT)
    display(df)
    print(f'Finding geolocations for {df.shape[0]} contacts')
    # 1 - convenient function to delay between geocoding calls
    geocoder = RateLimiter(locator.geocode, min_delay_seconds=delay_seconds)
    # 2- - create location column
    df['location'] = df['Address'].progress_apply(geocoder)
    # 3 - create longitude, laatitude and altitude from location column (returns tuple)
    df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
    # 4 - split point column into latitude, longitude and altitude columns
    df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)
    na_values = {'latitude': 0.0, 'longitude': 0.0, 'altitude': 0.0}
    
    return df.fillna(value=na_values)

def addresses_to_geolocations(addresses: list):
    # Takes a list of addresses and returns geolocation information
    # It looks in the cache file first and updates caches after a lookup
    cached_geolocations = read_cached_geolocations()

    if not cached_geolocations.empty:
        cached_subframe = cached_geolocations[cached_geolocations.Address.isin(addresses)]
        to_look_up = list(set(addresses) - set(cached_subframe.Address))
    else:
        to_look_up = list(set(addresses))
        
    if to_look_up:
        found_locations = find_geolocations(to_look_up)
        # Now update cached with info from the newly found geolocations
        new_cache = pd.concat([cached_geolocations, found_locations], 
                  ignore_index=True).drop_duplicates(subset=['Address'], keep='last')
        new_cache.to_csv(geolocation_cache_path, index=False)
    else:
        # Everything already in the cache
        new_cache = cached_geolocations
        
    return new_cache[new_cache.Address.isin(addresses)]

In [ ]:
# Call with prepare_unique_lists(birders_df2)
def prepare_unique_lists(df):
    # Create a list for each person of species that only they will see
    dfobs, omdf = pd.DataFrame(), pd.DataFrame()
    try:
        only_me = []
        dfobs = df[[not x.empty for x in list(df.observations)]].copy()
        dfobs.reset_index(drop=True, inplace=True)
        obslist = list(dfobs['observations'])
        print(f'Rows with observations: {len(obslist)}')

        species = [list(row['comName']) for row in obslist]
        for index in range(len(obslist)):
            mine = set(species[index])
            bna = list(flatten(species[:index]+species[index+1:]))
            others = set(bna)
            mo = list(mine - others)
            only_me.append(sorted(mo))

        # Pad lists to max len to fix the "raggedness"
        N = max([len(x) for x in only_me])
        only_me_padded = [list(more_itertools.padded(x, '', N)) for x in only_me]
        omdf = pd.DataFrame(only_me_padded).T
        omdf.columns=dfobs.Name
    except Exception as ee:
        print('prepare_unique_lists', ee)
        [print(type(x)) for x in list(df.observations)]

    return dfobs, omdf

In [ ]:
def generate_5mr_reports(contacts_path = contacts_base_path, unique_path = unique_species_path,
                        participants = [], circle_radius = DEFAULT_CIRCLE_RADIUS, days_back = DEFAULT_DAYS_BACK):
    participant_observations, contacts = None, None
    try:
        contacts = load_and_update_contacts(contacts_path)
        pcontacts = filter_participants(contacts, participants)
        participant_observations = get_observations(pcontacts, circle_radius, days_back)

        dfobs, omdf = prepare_unique_lists(participant_observations)

        ts = f'{pd.Timestamp.now().replace(microsecond=0)}'
        prefix = MY5MR_PREFIX + re.sub(r'[ ]', '_', re.sub(r'[:]', '-', ts)) + '_'
        for index, row in dfobs.iterrows():
            fname = prefix + re.sub(r'[ ]', '',row.Name) + '.csv'
            row['observations'].to_csv(reports_path / fname)

        # Now write out single CSV with only_me data
        omdf.to_csv(unique_path)
    except Exception as ee:
        print(ee)
        traceback.print_tb(ee.__traceback__)
        raise
        
    return participant_observations, contacts

## Make Map

In [ ]:
def create_circles_map(group_center_pt, radius_in_miles = DEFAULT_CIRCLE_RADIUS):
    # See e.g. https://github.com/python-visualization/folium/blob/master/folium/vector_layers.py
    # color is e.g. '#3388ff' or 'red'

    # radius (float) – Radius of the circle, in meters.
    circle_radius = 1000 * miles_to_kilometers(radius_in_miles)

    # Web colors. Drop 'maroon', 'olive' ebcause they look terrible on map
    circle_colors = ['red', 'yellow', 'lime', 'green', 'aqua', 'teal', 'blue', 'navy', 'fuchsia', 'purple']
    circle_colors_count = len(circle_colors)

    mm = folium.Map(location=group_center_pt, zoom_start=11)
    for ix, row in df5mr.iterrows():
        location = (float(row['latitude']), float(row['longitude']))
    #     print(location)
        _ = mm.add_child(folium.vector_layers.Circle(location, circle_radius, stroke=1, 
                                                     color=circle_colors[ix % circle_colors_count], 
                                                     opacity=0.5, dash_array='4 1'))
    _ = mm.add_child(folium.LayerControl())
    mm.save(outfile=sharpies_map_path.as_posix())
    return mm

## Hotspots

In [ ]:
from io import StringIO

def get_hotspots_for_region(regionCode: str) -> pd.DataFrame:
    # https://ebird.org/ws2.0/ref/hotspot/geo?lat=37.4407&lng=-122.0936&fmt=json&dist=6
    # https://api.ebird.org/v2/ref/hotspot/{{regionCode}}
    headers = ['locid', 'r1', 'r2', 'r3', 'lat', 'lng', 'name', 'date', 'num']
    results = pd.DataFrame()
    try:
        api_url_base = 'https://api.ebird.org/v2/ref/hotspot'
        api_auth_header = {'X-eBirdApiToken': ebird_api_key}
        url = f'{api_url_base}/{regionCode}'
        params = None #{ 'maxResults' : 200}
        rr = requests.get(url, stream=True) #params=params, headers=api_auth_header, 
        if rr.status_code == requests.codes.ok:
            results = pd.read_csv(StringIO(rr.text), names = headers, index_col=False)
        rr.raise_for_status()

    except Exception as ee:
        print(ee)

    return results

def get_hotspots_for_region_cached(regionCode: str) -> pd.DataFrame:
    fpath = processed_data_path / f'hotspots-{regionCode}.csv'
    if not fpath.is_file():
        hs_df = get_hotspots_for_region(regionCode)
        hs_df.to_csv(fpath, index=False)
        return hs_df
    else:
        return pd.read_csv(fpath, index_col=False)

In [ ]:
def hotspot_data_for_regions(region_codes: list):
    first_region, *remaining_regions = region_codes
    combined = get_hotspots_for_region_cached(first_region)

    for rc in remaining_regions:
        region_hotspots = get_hotspots_for_region_cached(rc)
        combined = pd.concat([combined, region_hotspots], ignore_index=True) #, ignore_index=True

    # Some hotspots have NAN (e.g. Ocean View Park (ALA Co.))
    combined['num'].fillna(0, inplace=True)

    combined_hotspot_geometry = [Point(x, y) for x, y in zip(combined.lng, combined.lat)]
    combined_hotspot_gdf = gpd.GeoDataFrame(combined, geometry=combined_hotspot_geometry)
    center_pt = combined_hotspot_gdf.unary_union.convex_hull.centroid.coords[0][::-1]

    return combined, combined_hotspot_gdf, center_pt

### Table with all hotspots for each person

In [ ]:
def all_hotspots_for_each_person(df5mr, hotspots_gdf, circle_radius_miles = DEFAULT_CIRCLE_RADIUS):
#     hotspots_gdf - A geopandas.GeoDataFrame made from (lat,lng) tuples for each hotspot in the county
    birder_geometry = [Point(x, y) for x, y in zip(df5mr.longitude.astype(float), df5mr.latitude.astype(float))]
    birder_gdf = gpd.GeoDataFrame(df5mr, geometry=birder_geometry)

    individual_hotspots = []
    circle_radius = 1000*miles_to_kilometers(circle_radius_miles) # radius in meters  1000 * 
    # combined_hotspot_tuples = zip(hotspots_gdf.name, hotspots_gdf.geometry)
    # Assumes geometry is single point
    for birder_loc in birder_gdf.geometry:
        combined_hotspot_tuples = zip(list(hotspots_gdf.name), list(hotspots_gdf.geometry))
        hotspots = [name for name, pt2 in combined_hotspot_tuples if geog.distance(pt2, birder_loc) < circle_radius]
        individual_hotspots.append(hotspots)
        
    # Pad lists to max len to fix the "raggedness"
    N = max([len(x) for x in individual_hotspots])
    individual_hotspots_padded = [list(more_itertools.padded(x, '', N)) for x in individual_hotspots]
    ihdf = pd.DataFrame(individual_hotspots_padded).T
    ihdf.columns=birder_gdf.Name
    ihdf.to_csv(hotspots_path, index=False)
    
    return ihdf, individual_hotspots

### Table with any hotspots unique to each person

In [ ]:
def unique_hotspots_for_each_person(df5mr, individual_hotspots, circle_radius_miles = DEFAULT_CIRCLE_RADIUS):
#     individual_hotspots - Returned from call to all_hotspots_for_each_person
    only_my_hotspots = []
    for index in range(len(individual_hotspots)):
        mine = set(individual_hotspots[index])
        bna = list(flatten(individual_hotspots[:index]+individual_hotspots[index+1:]))
        others = set(bna)
        mo = list(mine - others)
        only_my_hotspots.append(sorted(mo))
        
    # Pad lists to max len to fix the "raggedness"
    N = max([len(x) for x in only_my_hotspots])
    only_my_hotspots_padded = [list(more_itertools.padded(x, '', N)) for x in only_my_hotspots]
    omhdf = pd.DataFrame(only_my_hotspots_padded).T
    omhdf.columns=birder_gdf.Name
    omhdf.to_csv(unique_hotspots_path, index=False)
        
    return omhdf

## Limited Observers Table

In [ ]:
def create_limited_observers_table(df5mr, max_observers = DEFAULT_MAX_OBSERVERS):
    '''
    Each row in df5mr has an individual observer (Name) and a DataFrame with all observations
    in their 5MR (observations). In the observations DataFrame we us the species name "comName"
    and the location (i.e. hotspot) name "locName"
    The output is a table where each row is a species that has only been seen in the 5MRs 
    for "max_observers" people or less. We use "limited" to denote these
    '''
    species_dict = {}
    for ix, row in df5mr.iterrows():
        # For each individual observation (for this person)
        for jx, obs in row.observations.iterrows():
            key = obs.comName
            cur = species_dict.get(key, [])
            val = (row.Name, obs.locName)
            cur.append(val)
            species_dict[key] = cur

    limited_dict = dict(filter(lambda elem: len(elem[1]) <= max_observers, species_dict.items()))

    limiteds = []
    for key in sorted(limited_dict.keys()):
        item = limited_dict[key]
        line = [key]
        for x,y in item:
            line.append(x)
            line.append(y)
        limiteds.append(line)

    cols = ['Species']
    for ix in range(max_observers):
        cols.append(f'Name {ix+1}')
        cols.append(f'Location {ix+1}')
    
    limited_df = pd.DataFrame(limiteds, columns = cols)
    limited_df.to_csv(limited_path, index=False)
    
    return limited_df

## Individual Hotspot Maps

In [ ]:
srgb_black  = '#000000'

marker_color_names = [
    'HotPink', 'Red', 'Orange', 'Yellow', 'Maroon', 'Plum',
    'Magenta', 'BlueViolet', 'Indigo', 'Lime', 'ForestGreen', 'Cyan',
    'Navy', 'Blue', 'Teal', 'Silver', 'cornflowerblue',
    'indianred', 'indigo', 'ivory', 'khaki', 'lavender', 'lavenderblush', 'lawngreen', 
    'lemonchiffon', 'lightblue', 'lightcoral', 'lightcyan', 'lightgoldenrodyellow', 
    'lightgray', 'lightgrey', 'lightgreen', 'lightpink', 'lightsalmon', 'lightseagreen', 'lightskyblue'    
]

marker_colors = dict([(colorname, webcolors.name_to_hex(colorname.lower())) for colorname in marker_color_names])


def color_for_marker(marker_size):
    colors = [marker_colors.get(mn, 'HotPink') for mn in ['lightgrey', 'lightcyan', 'cornflowerblue']]
    return colors[marker_size-1]


def create_hotspots_markers(hotspots_df):
    # hotspots_df has columns ['locid', 'lat', 'lng', 'name', 'num', 'marker_size']
    markers = folium.FeatureGroup(name="Markers")

    for index, row in hotspots_df.iterrows():
        hname = row['name']
        xpopup = folium.Popup(row.to_frame().to_html())
        marker = folium.Marker(
            location=[row['lat'], row['lng']],
            popup=xpopup,
            tooltip=hname,
            icon=folium.plugins.BeautifyIcon(
                icon='twitter',
                icon_shape='marker',
                text_color=srgb_black,  # actually icon color
                background_color=color_for_marker(row['marker_size']),
                border_width=row['marker_size']
            )
        )

        markers.add_child(marker)

    return markers


def my_hotspots_map(my_center_pt, my_hotspots_df, my_path, radius_in_miles = 5):
    # See e.g. https://github.com/python-visualization/folium/blob/master/folium/vector_layers.py
    # color is e.g. '#3388ff' or 'red'

    # radius (float) – Radius of the circle, in meters.
    circle_radius = 1000 * miles_to_kilometers(radius_in_miles)

    # Web colors. Drop 'maroon', 'olive' ebcause they look terrible on map
    circle_colors = ['red', 'yellow', 'lime', 'green', 'aqua', 'teal', 'blue', 'navy', 'fuchsia', 'purple']
    circle_colors_count = len(circle_colors)

    mm = folium.Map(location=my_center_pt, zoom_start=11)
    
    # Draw the circle
    _ = mm.add_child(folium.vector_layers.Circle(my_center_pt, circle_radius, stroke=1, 
                                                     color='lime', 
                                                     opacity=0.5, dash_array='4 1'))
    
    _ = mm.add_child(create_hotspots_markers(my_hotspots_df))
    _ = mm.add_child(folium.LayerControl())
    mm.save(outfile=my_path.as_posix())
    
    return mm


def individual_hotspot_maps(df5mr):
    for my_name in df5mr.Name:
        my_entry = df5mr[df5mr.Name==my_name].iloc[0] #['latitude', 'longitude']
        my_center_pt = my_entry['latitude'], my_entry['longitude']

        my_hotspots = sorted(list(set(ihdf[my_name])))
        my_hotspots_df = hotspots[hotspots.name.isin(my_hotspots)].reset_index(drop=True).copy()[['locid', 'lat', 'lng', 'name', 'num']]

        my_hotspots_df['marker_size'] = pd.qcut(my_hotspots_df['num'], q=3, labels=[1, 2, 3])

        my_path = reports_path / f'{MY5MR_PREFIX}map_{my_name}.html'
        my_radius_in_miles = 5

        mm = my_hotspots_map(my_center_pt, my_hotspots_df, my_path, my_radius_in_miles)


## Determine Region Codes from Contacts List

In [ ]:
def download_us_county_shape_files():
    #https://catalog.data.gov/dataset/tiger-line-shapefile-2017-nation-u-s-current-county-and-equivalent-national-shapefile
    
    try:
        shape_zip_url = 'https://www2.census.gov/geo/tiger/TIGER2017/COUNTY/tl_2017_us_county.zip'
        xruxidownload.download_file(shape_zip_url, shape_zip_path.as_posix(), False)   

        shape_dest_dir = processed_data_path / 'tl_2017_us_county'
        shape_dest_dir.mkdir(mode=0o744, parents=False, exist_ok=False)

        # https://thispointer.com/python-how-to-unzip-a-file-extract-single-multiple-or-all-files-from-a-zip-archive/
        with ZipFile(shape_zip_path, 'r') as zipObj:
           # Extract all the contents of zip file in different directory
           zipObj.extractall(shape_dest_dir)
    
    except Exception as ee:
        print(ee)
        traceback.print_tb(ee.__traceback__)
        raise UserWarning(f'Could not download US County shape files from: {shape_zip_url}')

In [ ]:
# https://www2.census.gov/geo/tiger/TIGER2017/COUNTY/tl_2017_us_county.zip
def load_us_county_data() -> (pd.DataFrame, pd.DataFrame):
    # Load map data for counties in US. This takes a bit of time
    # Both files are required, 
    map_us_counties, state_fips_master_df = pd.DataFrame(), pd.DataFrame() # Failure mode
    
    if not shape_dest_dir.is_dir():
        download_us_county_shape_files()
    
    try:
        map_us_counties = gpd.read_file(county_shx_path)
    except Exception as ee:
        print(ee)
        raise UserWarning(f'Missing US County data: {county_shx_path}')

    try:
        state_fips_master_df = pd.read_csv(state_fips_master_path, dtype=str).fillna('')
    except Exception as ee:
        print(ee)
        raise UserWarning(f'Missing US State FIPS data: {state_fips_master_path}')

    return map_us_counties, state_fips_master_df

In [ ]:
# regionType: The region type: 'country', 'subnational1' or 'subnational2'.
# parentRegionCode: The country or subnational1 code, or 'world'.
# fmt: The format for the records, either 'csv' or 'json' (the default).
# https://api.ebird.org/v2/ref/region/list/{{regionType}}/{{parentRegionCode}}.{{fmt}}

def get_sub_region_list(parent_region_code: str, region_type: str):
    # https://api.ebird.org/v2/ref/region/list/{{regionType}}/{{parentRegionCode}}.{{fmt}}
    # e.g. get_sub_region_list('US', 'subnational1') for list of states
    # e.g. get_sub_region_list('US-CA', 'subnational2') for county list

    result = pd.DataFrame()
    try:
        fmt = 'json'
        api_url = f'https://api.ebird.org/v2/ref/region/list/{region_type}/{parent_region_code}.{fmt}'
        api_auth_header = {'X-eBirdApiToken': ebird_api_key}

        rr = requests.get(api_url, params=None, headers=api_auth_header, stream=True)
        if rr.status_code == requests.codes.ok:
            result = pd.DataFrame(rr.json())
        rr.raise_for_status()

    except Exception as ee:
        print(ee)

    return result


def region_codes_from_contacts(contacts: pd.DataFrame, map_us_counties, state_fips_master_df) -> pd.DataFrame:

    def matching_row(row):
        # row from srl has code	name like US-SC-001	Abbeville
        return (row.code[3:5], row.xname) in list(zip(i2.state2, i2.NAME))

    def get_2char_state_code(fips):
        return str(state_fips_master_df[state_fips_master_df.fips==str(fips)].state_abbr.values[0])

    region_codes_df = pd.DataFrame()
    
    contacts_geometry = [Point(x, y) for x, y in zip(contacts.longitude.astype(float), contacts.latitude.astype(float))]
    contacts_gdf = gpd.GeoDataFrame(contacts, geometry=contacts_geometry)

    contacts_gdf_x = gpd.GeoDataFrame(contacts_gdf, geometry='geometry',crs={'init' :'epsg:4269'})

    contacts_gdf_y = contacts_gdf_x.copy()
    contacts_gdf_y['geometry']=contacts_gdf_x['geometry'].buffer(5*(1/60))

    intersections_df = pd.DataFrame()
    for geo in contacts_gdf_y.geometry:
        intersections_df = pd.concat([intersections_df, map_us_counties[map_us_counties.intersects(geo)]])

    srl = get_sub_region_list('US', 'subnational2')
    # Rename 'name' to 'xname' so we can use srl.xname (srl.name is reserved)
    srl.columns = ['code', 'xname']

    intersections_df['state2'] = intersections_df.STATEFP.astype(int).apply(get_2char_state_code)
    i2 = intersections_df.copy()[['NAME', 'state2']].drop_duplicates().reset_index(drop=True)
    region_codes_df = srl[srl.apply(matching_row, axis=1)].reset_index(drop=True)

    return region_codes_df

def get_region_codes_for_contacts(contacts, params) -> list:
    # If the parameters file exists and has a list of region codes, use that
    if not params.empty:
        region_codes = [y for y in [x.strip() for x in params.Region] if y!='']
        if region_codes:
            return region_codes
    
    region_codes = []
    map_us_counties, state_fips_master_df = load_us_county_data()

    region_codes_df = region_codes_from_contacts(contacts, map_us_counties, state_fips_master_df)

    return list(region_codes_df.code)


def get_participants_from_params(params) -> list:
    if not params.empty:
        participants = [y for y in [x.strip() for x in params.Participants] if y!='']
        return participants
    else:
        participants = [] # really all
        

def get_numeric_from_params(params, column_name, defaultval):
    val = defaultval
    try:
        if not params.empty:
            val = params[column_name].values[0]
    except Exception as ee:
        pass
    
    return val

# Initialization

In [ ]:
# Initializations
credentials = xutilities.load_credentials(eBirdCredential_path)['credentials']
ebird_api_key = credentials['api_key']

# For geocoder progress bar (see progress_apply)
tqdm.pandas()    

# Main

In [ ]:
if __name__ == '__main__':
    
    create_project_paths()
    
    try:
        parameters_path = parameters_base_path
        if DEBUGGING:
            # Comment out one by one
            sample_path = base_path / 'SampleInputs'
            parameters_path = sample_path / 'Parameters-Default.xlsx'
            parameters_path = sample_path / 'Parameters-Regions.xlsx'
#             parameters_path = sample_path / 'Parameters-Participants.xlsx'

        params = read_parameters(parameters_path)
        maxobservers = int(get_numeric_from_params(params, 'MaxObservers', DEFAULT_MAX_OBSERVERS))
        circle_radius = float(get_numeric_from_params(params, 'Radius', DEFAULT_CIRCLE_RADIUS))
        days_back = int(get_numeric_from_params(params, 'DaysBack', DEFAULT_DAYS_BACK))
        if days_back >= 0 or days_back > MAX_DAYS_BACK:
            days_back = DEFAULT_DAYS_BACK
        
        contacts_path = contacts_base_path
        if DEBUGGING:
            # Comment out one by one
            sample_path = base_path / 'SampleInputs'
            contacts_path = sample_path / 'Contacts-Sample-crosscountry.xlsx'
            contacts_path = sample_path / 'Contacts-Sample-missing.xlsx'
            contacts_path = sample_path / 'Contacts-Sample.xlsx'

        participants = get_participants_from_params(params)
        df5mr, contacts = generate_5mr_reports(contacts_path, unique_species_path, 
                                               participants, circle_radius, days_back)
    
        region_codes = get_region_codes_for_contacts(contacts, params)
        print(f'Region codes: {region_codes}')
        hotspots, hotspot_gdf, hs_center_pt = hotspot_data_for_regions(region_codes)

        birder_geometry = [Point(x, y) for x, y in zip(df5mr.longitude.astype(float), df5mr.latitude.astype(float))]
        birder_gdf = gpd.GeoDataFrame(df5mr, geometry=birder_geometry)

        # Reverse the order of the point; Shapely uses (lng, lat) but folium is (lat, lng)
        # This point is used to center the map
        group_center_pt = birder_gdf.unary_union.convex_hull.centroid.coords[0][::-1]

        mm = create_circles_map(group_center_pt, circle_radius)

        ihdf, individual_hotspots = all_hotspots_for_each_person(df5mr, hotspot_gdf, circle_radius)
        uhdf = unique_hotspots_for_each_person(df5mr, individual_hotspots, circle_radius)
        limited_df = create_limited_observers_table(df5mr, maxobservers)

        individual_hotspot_maps(df5mr)
        
    except Exception as ee:
        print(ee)
        traceback.print_tb(ee.__traceback__)


    print('\nDone')